In [1]:
from itertools import combinations
import csv

def generateMFCS(MFCS, infrequent_itemsets):
	""" Generate the updated MFCS by modifing itemsets that have infrequent itemsets as its subset
	Parameters
	----------
	MFCS : list of lists
		The list of Maximal Frequent Candidate Sets
	infrequent_itemsets : list of lists
		The list of infrequent itemsets
	Returns
	-------
	lists of lists
		Updated MFCS
	"""

	MFCS = MFCS.copy()

	for infrequent_itemset in infrequent_itemsets:

		for MFCS_itemset in MFCS.copy():
			
			# If infrequent itemset is a subset of MFCS itemset
			if all(_item in MFCS_itemset for _item in infrequent_itemset):
				MFCS.remove(MFCS_itemset)
				
				for item in infrequent_itemset:
					updated_MFCS_itemset = MFCS_itemset.copy()
					updated_MFCS_itemset.remove(item)

					if not any(all(_item in _MFCS_itemset for _item in updated_MFCS_itemset) for _MFCS_itemset in MFCS):
						MFCS.append(updated_MFCS_itemset)

	return MFCS


def pruneCandidatesUsingMFS(candidate_itemsets, MFS):
	""" Prune the candidate itemsets that are subsets of MFS itemsets
	Parameters
	----------
	candidate_itemsets : lists of lists
		The list of candidate itemsets
	MFS : lists of lists
		The list of Maximal Frequent Itemsets
	Returns
	-------
	lists of lists
		The list of candidate itemsets with are not subsets of any itemset in MFS
	"""

	candidate_itemsets = candidate_itemsets.copy()

	for itemset in candidate_itemsets.copy():
		if any(all(_item in _MFS_itemset for _item in itemset) for _MFS_itemset in MFS):
			candidate_itemsets.remove(itemset)

	return candidate_itemsets


def generateCandidateItemsets(level_k, level_frequent_itemsets):
	""" Generate and prune the candidate itemsets for next level using the frequent itemsets of the current level 
	
	Parameters
	----------
	level_k : int
		The current level number
	level_frequent_itemsets : list of lists
		The list of frequent itemsets of current level
	Returns
	-------
	list of lists
		The candidate itemsets of the next level
	"""

	n_frequent_itemsets = len(level_frequent_itemsets)

	candidate_frequent_itemsets = []

	for i in range(n_frequent_itemsets):
		j = i+1
		while (j<n_frequent_itemsets) and (level_frequent_itemsets[i][:level_k-1] == level_frequent_itemsets[j][:level_k-1]):
			
			candidate_itemset = level_frequent_itemsets[i][:level_k-1] + [level_frequent_itemsets[i][level_k-1]] + [level_frequent_itemsets[j][level_k-1]]
			candidate_itemset_pass = False

			if level_k == 1:
				candidate_itemset_pass = True
				
			elif (level_k == 2) and (candidate_itemset[-2:] in level_frequent_itemsets):
				candidate_itemset_pass = True

			elif all((list(_)+candidate_itemset[-2:]) in level_frequent_itemsets for _ in combinations(candidate_itemset[:-2], level_k-2)):
				candidate_itemset_pass = True
				
			if candidate_itemset_pass:
				candidate_frequent_itemsets.append(candidate_itemset)

			j += 1

	return candidate_frequent_itemsets


def pruneCandidatesUsingMFCS(candidate_itemsets, MFCS):
	""" Prune the candidate itemsets that are not subsets of any itemsets in current MFCS 
	Parameters
	----------
	candidate_itemsets : lists of lists
		The list of candidate itemsets
	MFCS : lists of lists
		The list of Maximal Frequent Candidate Itemsets
	Returns
	-------
	lists of lists
		The list of candidate itemsets that are subsets of some itemsets in current MFCS 
	"""

	candidate_itemsets = candidate_itemsets.copy()

	for itemset in candidate_itemsets.copy():
		if not any(all(_item in _MFCS_itemset for _item in itemset) for _MFCS_itemset in MFCS):
			candidate_itemsets.remove(itemset)

	return candidate_itemsets


def pincerSearch(transactions, min_support):
	""" Extract the Maximal Frequent Itemsets (MFI) from the transactions 
	
	Parameters
	----------
	transactions : a list of sets
		The list of transactions
	min_support : int
		The minimum support for an itemset to be considered frequent
	Returns
	-------
	list of lists
		The list of MFS which contains all maximal frequent itemsets
	"""

	# Extract the list of items in the transactions
	items = set()
	for transaction in transactions:
		items.update(transaction)
	items = sorted(list(items))
	
	level_k = 1 # The current level number

	level_frequent_itemsets = [] # Level 0: Frequent itemsets
	candidate_frequent_itemsets = [[item] for item in items] # Level 1: Candidate itemsets
	level_infrequent_itemsets = [] # Level 0: Infrequent itemsets

	MFCS = [items.copy()] # Maximal Frequent Candidate Sets
	MFS = [] # Maximal Frequent Sets

	print("MFCS = {}".format(MFCS))
	print("MFS = {}\n".format(MFS))

	while candidate_frequent_itemsets:
		
		print("LEVEL {}: ".format(level_k))
		print("C{} = {}".format(level_k, candidate_frequent_itemsets))

		candidate_freq_itemsets_cnts = [0]*len(candidate_frequent_itemsets)
		MFCS_itemsets_cnts = [0]*len(MFCS)

		# Step 1: Read the database and count supports for Ck and MFCS
		for transaction in transactions:
			
			for i, itemset in enumerate(candidate_frequent_itemsets):
				if all(_item in transaction for _item in itemset):
					candidate_freq_itemsets_cnts[i] += 1

			for i, itemset in enumerate(MFCS):
				if all(_item in transaction for _item in itemset):
					MFCS_itemsets_cnts[i] += 1

		for itemset, support in zip(candidate_frequent_itemsets, candidate_freq_itemsets_cnts):
			print("{} -> {}".format(itemset, support), end=', ')
		print()

		for itemset, support in zip(MFCS, MFCS_itemsets_cnts):
			print("{} -> {}".format(itemset, support), end=', ')
		print()

		# Step 2: MFS := MFS U {frequent itemsets in MFCS}
		MFS.extend([itemset for itemset, support in zip(MFCS, MFCS_itemsets_cnts) if ((support >= min_support) and (itemset not in MFS))])
		print("MFS = {}".format(MFS))

		# Step 3: Sk := {infrequent itemsets in Ck}
		level_frequent_itemsets = [itemset for itemset, support in zip(candidate_frequent_itemsets, candidate_freq_itemsets_cnts) if support >= min_support]
		level_infrequent_itemsets = [itemset for itemset, support in zip(candidate_frequent_itemsets, candidate_freq_itemsets_cnts) if support < min_support]

		print("L{} = {}".format(level_k, level_frequent_itemsets))
		print("S{} = {}".format(level_k, level_infrequent_itemsets))

		# Step 4: call MFCS-gen algorithm if Sk != NULL
		MFCS = generateMFCS(MFCS, level_infrequent_itemsets)
		print("MFCS = {}\n".format(MFCS))

		# Step 5: call MFS-pruning procedure
		level_frequent_itemsets = pruneCandidatesUsingMFS(level_frequent_itemsets, MFS)

		# Step 6: Generate candidates Ck+1 from Ck (using generate and prune)
		candidate_frequent_itemsets = generateCandidateItemsets(level_k, level_frequent_itemsets)

		# Step 7: If any frequents itemsets in Ck is removed in MFS-pruning procedure
		# Call the recovery procedure to recover candidates to Ck+1

		# Step 8: call MFCS-prune procedure to prune candidates in Ck+1
		candidate_frequent_itemsets = pruneCandidatesUsingMFCS(candidate_frequent_itemsets, MFCS)

		# Step 9: k := k+1
		level_k += 1

	return MFS

if __name__ == '__main__':
    transactions = list(csv.reader(open(r'H:\store_data.csv')))
    MFS = pincerSearch(transactions, 100)
    print("MFS = {}".format(MFS))


MFCS = [[' asparagus', 'almonds', 'antioxydant juice', 'asparagus', 'avocado', 'babies food', 'bacon', 'barbecue sauce', 'black tea', 'blueberries', 'body spray', 'bramble', 'brownies', 'bug spray', 'burger sauce', 'burgers', 'butter', 'cake', 'candy bars', 'carrots', 'cauliflower', 'cereals', 'champagne', 'chicken', 'chili', 'chocolate', 'chocolate bread', 'chutney', 'cider', 'clothes accessories', 'cookies', 'cooking oil', 'corn', 'cottage cheese', 'cream', 'dessert wine', 'eggplant', 'eggs', 'energy bar', 'energy drink', 'escalope', 'extra dark chocolate', 'flax seed', 'french fries', 'french wine', 'fresh bread', 'fresh tuna', 'fromage blanc', 'frozen smoothie', 'frozen vegetables', 'gluten free bar', 'grated cheese', 'green beans', 'green grapes', 'green tea', 'ground beef', 'gums', 'ham', 'hand protein bar', 'herb & pepper', 'honey', 'hot dogs', 'ketchup', 'light cream', 'light mayo', 'low fat yogurt', 'magazines', 'mashed potato', 'mayonnaise', 'meatballs', 'melons', 'milk', 'mi

['almonds', 'avocado'] -> 13, ['almonds', 'black tea'] -> 7, ['almonds', 'brownies'] -> 5, ['almonds', 'burgers'] -> 39, ['almonds', 'butter'] -> 6, ['almonds', 'cake'] -> 23, ['almonds', 'carrots'] -> 7, ['almonds', 'cereals'] -> 8, ['almonds', 'champagne'] -> 5, ['almonds', 'chicken'] -> 18, ['almonds', 'chocolate'] -> 45, ['almonds', 'cookies'] -> 9, ['almonds', 'cooking oil'] -> 13, ['almonds', 'cottage cheese'] -> 6, ['almonds', 'eggs'] -> 49, ['almonds', 'energy bar'] -> 6, ['almonds', 'energy drink'] -> 6, ['almonds', 'escalope'] -> 10, ['almonds', 'french fries'] -> 33, ['almonds', 'french wine'] -> 10, ['almonds', 'fresh bread'] -> 14, ['almonds', 'fresh tuna'] -> 12, ['almonds', 'fromage blanc'] -> 6, ['almonds', 'frozen smoothie'] -> 21, ['almonds', 'frozen vegetables'] -> 23, ['almonds', 'grated cheese'] -> 13, ['almonds', 'green tea'] -> 38, ['almonds', 'ground beef'] -> 29, ['almonds', 'gums'] -> 3, ['almonds', 'ham'] -> 6, ['almonds', 'herb & pepper'] -> 10, ['almonds', 

In [2]:
import pandas as pd
from mlxtend.frequent_patterns import apriori
from mlxtend.preprocessing import TransactionEncoder
from mlxtend.frequent_patterns import association_rules

In [3]:
te = TransactionEncoder()
te_ary = te.fit(transactions).transform(transactions)
df = pd.DataFrame(te_ary, columns=te.columns_)
frequent_itemsets = apriori(df, min_support=0.013, use_colnames=True)

print(frequent_itemsets)


      support                                 itemsets
0    0.020397                                (almonds)
1    0.033329                                (avocado)
2    0.014265                              (black tea)
3    0.033729                               (brownies)
4    0.087188                                (burgers)
5    0.030129                                 (butter)
6    0.081056                                   (cake)
7    0.015331                                (carrots)
8    0.025730                                (cereals)
9    0.046794                              (champagne)
10   0.059992                                (chicken)
11   0.163845                              (chocolate)
12   0.080389                                (cookies)
13   0.051060                            (cooking oil)
14   0.031862                         (cottage cheese)
15   0.013198                               (eggplant)
16   0.179709                                   (eggs)
17   0.027